In [3]:
import os, sys

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print("PROJECT_ROOT:", PROJECT_ROOT)


PROJECT_ROOT: e:\antibody-seq-developability\antibody-seq-developability


In [ ]:
from src.data_oas import inspect_oas_columns, load_oas_human_paired

inspect_oas_columns()
df = load_oas_human_paired()
df.head()


Resolving data files:   0%|          | 0/170 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/170 [00:00<?, ?it/s]

['data_unit', 'sequence_alignment_aa_heavy', 'cdr1_aa_heavy', 'cdr2_aa_heavy', 'cdr3_aa_heavy', 'sequence_alignment_aa_light', 'cdr1_aa_light', 'cdr2_aa_light', 'cdr3_aa_light']


Resolving data files:   0%|          | 0/170 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/170 [00:00<?, ?it/s]

ValueError: None of the candidate columns for isotype were found: ['isotype', 'c_call_heavy', 'heavy_isotype']